In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import skew, kurtosis
plt.style.use('fivethirtyeight')

In [2]:
CRSP_stocks = pd.read_csv('QAM1.csv',header = 0)

In [3]:
def stocks(data):
    #data = CRSP_stocks.copy()
    data = data.copy()
    data = data[(data.SHRCD == 10) | (data.SHRCD == 11)]
    data = data[(data.EXCHCD == 1) | (data.EXCHCD == 2) | (data.EXCHCD == 3)]
    data.date = data['date'].astype(str)
    data.date = pd.to_datetime(data.date)
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year
    data['RET'] =pd.to_numeric(data.RET,errors = 'coerce')
    data['DLRET'] = pd.to_numeric(data.DLRET, errors = 'coerce')
    data['Stock_lag_MV'] = abs(data['PRC'] * data['SHROUT'])
    data['Stock_lag_MV'] = data.groupby('PERMNO')['Stock_lag_MV'].shift(1)
    data['CumReturns'] = 0
    data.loc[(data.RET.notna() & data.DLRET.notna() == True),'CumReturns'] = ((1 + data.loc[(data.RET.notna() & data.DLRET.notna() == True),'RET'])*(1 + data.loc[(data.RET.notna() & data.DLRET.notna() == True),'DLRET'])-1)
    data.loc[(data.RET.isna() & data.DLRET.notna() == True),'CumReturns'] = data.DLRET
    data.loc[(data.RET.notna() & data.DLRET.isna() == True),'CumReturns'] = data.RET
    data['weights'] = data.groupby(['year','month'])['Stock_lag_MV'].transform(lambda x: (x/np.sum(x)))
    data['Vw'] = data['weights']*data['CumReturns']
    Data_1 = pd.DataFrame()
    Data_1['Stock_lag_MV'] = data.groupby(['year','month'])['Stock_lag_MV'].sum()
    Data_1 = Data_1.reset_index()
    Data_1['Stock_Ew_Ret'] = data.groupby(['year','month'])['CumReturns'].transform(lambda x: np.nanmean(x))
    Data_1['Stock_Vw_Ret'] = data.groupby(['year','month'])['Vw'].transform(lambda x: np.nansum(x))
    Data_1 = Data_1[Data_1['year']>=1926]
    Data_1 = Data_1[['year','month','Stock_lag_MV','Stock_Ew_Ret','Stock_Vw_Ret']]
    return Data_1

In [4]:
Monthly_CRSP_Stocks = stocks(CRSP_stocks).reset_index(drop=True)
Monthly_CRSP_Stocks

,year,month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret
0,1926,1,2.690395e+07,0.043403,0.012096
1,1926,2,2.703235e+07,0.062274,0.076467
2,1926,3,2.616208e+07,0.047576,0.054730
3,1926,4,2.450693e+07,0.014428,-0.007594
4,1926,5,2.527439e+07,0.036977,0.051010
...,...,...,...,...,...
1123,2019,8,2.959421e+10,0.035354,0.008429
1124,2019,9,2.877844e+10,0.084942,0.057586
1125,2019,10,2.918058e+10,0.010819,0.018459
1126,2019,11,2.974053e+10,-0.063984,-0.047392


In [5]:
CRSP_Bonds = pd.read_csv('Bonddata.csv')
CRSP_Bonds

,KYTREASNO,KYCRSPID,CRSPID,MCALDT,TMRETNUA,TMTOTOUT
0,204104,19260315.204750,19260315.204750,1926-01-30,0.002951,NaN
1,204104,19260315.204750,19260315.204750,1926-02-27,0.002768,NaN
2,204105,19260615.303000,19260615.303000,1926-01-30,0.004042,NaN
3,204105,19260615.303000,19260615.303000,1926-02-27,0.003237,NaN
4,204105,19260615.303000,19260615.303000,1926-03-31,0.002778,NaN
...,...,...,...,...,...,...
165890,205996,99990401.902000,99990401.902000,1935-01-31,0.011021,NaN
165891,205996,99990401.902000,99990401.902000,1935-02-28,0.002153,NaN
165892,205996,99990401.902000,99990401.902000,1935-03-30,-0.005593,NaN
165893,205996,99990401.902000,99990401.902000,1935-04-30,0.000295,NaN


In [6]:
def bonds(data_bond):
    #data_bond = CRSP_Bonds.copy()
    data_bond = data_bond.copy()
    data_bond['MCALDT'] = data_bond['MCALDT'].astype(str)
    data_bond['MCALDT'] = pd.to_datetime(data_bond['MCALDT'])
    data_bond['month'] = data_bond['MCALDT'].dt.month
    data_bond['year'] = data_bond['MCALDT'].dt.year
    data_bond['Bond_lag_MV'] = data_bond['TMTOTOUT']
    data_bond['Bond_lag_MV'] = data_bond.groupby('KYCRSPID')['Bond_lag_MV'].shift(1)
    data_bond['weights'] = data_bond.groupby(['MCALDT'])['Bond_lag_MV'].transform(lambda x: (x/np.nansum(x)))
    data_bond['Vw'] = data_bond['weights']*data_bond['TMRETNUA']
    Data_bond = pd.DataFrame()
    Data_bond['Bond_lag_MV'] = data_bond.groupby(['year','month'])['Bond_lag_MV'].sum()
    Data_bond = Data_bond.reset_index()
    Data_bond['Bond_Ew_Ret'] = data_bond.groupby(['year','month'])['TMRETNUA'].transform(lambda x: np.nanmean(x))
    Data_bond['Bond_Vw_Ret'] = data_bond.groupby(['year','month'])['Vw'].transform(lambda x: np.nansum(x))
    Data_bond = Data_bond[Data_bond['year']>=1926]
    Data_bond = Data_bond[['year','month','Bond_lag_MV','Bond_Ew_Ret','Bond_Vw_Ret']]
    return Data_bond

In [7]:
Monthly_CRSP_Bonds = bonds(CRSP_Bonds)
Monthly_CRSP_Bonds

,year,month,Bond_lag_MV,Bond_Ew_Ret,Bond_Vw_Ret
0,1926,1,0.0,0.005101,0.000000
1,1926,2,809.0,0.003621,0.006154
2,1926,3,809.0,0.005101,0.000000
3,1926,4,809.0,0.003621,0.006154
4,1926,5,809.0,0.003812,0.003881
...,...,...,...,...,...
1123,2019,8,14764084.0,-0.003169,-0.008545
1124,2019,9,14827349.0,-0.005543,-0.016599
1125,2019,10,15010773.0,0.006541,0.014576
1126,2019,11,15294120.0,0.003249,0.009573


In [8]:
Monthly_CRSP_Riskless = pd.read_csv('Treasurydata.csv')
Monthly_CRSP_Riskless

,caldt,t90ret,t30ret
0,19260130,0.004042,0.002951
1,19260227,0.003237,0.002768
2,19260331,0.002778,0.002778
3,19260430,0.003072,0.003072
4,19260528,0.001965,0.000342
...,...,...,...
1123,20190830,0.001762,0.001908
1124,20190930,0.001968,0.001787
1125,20191031,0.001961,0.001531
1126,20191129,0.001130,0.001197


In [9]:
def universe(Stocks, Bonds, Riskless):
    Stocks = Stocks.copy()
    Bonds = Bonds.copy()
    Riskless = Riskless.copy()
    
    Riskless['caldt'] = Riskless['caldt'].astype(str)
    Riskless['caldt'] = pd.to_datetime(Riskless['caldt'])
    Riskless['month'] = Riskless['caldt'].dt.month
    Riskless['year'] = Riskless['caldt'].dt.year
    
    Data = Stocks[['year','month','Stock_lag_MV']]
    temp = Stocks.merge(Riskless, on=["year","month"])
    temp['Stock_Excess_Vw_Ret'] = temp['Stock_Vw_Ret'] - temp['t90ret']
    Data['Stock_Excess_Vw_Ret'] = temp['Stock_Excess_Vw_Ret']
    
    temp2 = Bonds.merge(Riskless, on=["year","month"])
    temp2['Bond_Excess_Vw_Ret']= temp2['Bond_Vw_Ret'] - temp2['t30ret']
    
    Data['Bond_lag_MV'] = temp2['Bond_lag_MV']
    Data['Bond_Excess_Vw_Ret'] = temp2['Bond_Excess_Vw_Ret']
    return(Data)


In [10]:
Monthly_CRSP_Universe = universe(Monthly_CRSP_Stocks,Monthly_CRSP_Bonds,Monthly_CRSP_Riskless)

C:\Users\idhru\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
def port_returns(data):
    data = data.copy()
    data['Total_MV'] = data['Stock_lag_MV'] + data['Bond_lag_MV']
    data['Excess_Vw_Ret']= ((data['Stock_Excess_Vw_Ret']*data['Stock_lag_MV']) + (data['Bond_lag_MV']*data['Bond_Excess_Vw_Ret']))/(data['Total_MV'])
    #data.loc[0,'Excess_Vw_Ret'] = 0
    data['Excess_60_40_Ret'] = 0.6*data['Stock_Excess_Vw_Ret'] + 0.4*data['Bond_Excess_Vw_Ret']
    data['Stock_inverse_sigma_hat'] = (1/data['Stock_Excess_Vw_Ret'].transform(lambda x: x.shift(1).rolling(window=36).std()))
    data['Bond_inverse_sigma_hat'] = (1/data['Bond_Excess_Vw_Ret'].transform(lambda x: x.shift(1).rolling(window=36).std())) 
    data['Unlevered_k'] = 1/(data['Stock_inverse_sigma_hat'] + data['Bond_inverse_sigma_hat'] )
    data['Excess_Unlevered_RP_Ret'] = data['Unlevered_k']*data['Stock_inverse_sigma_hat']*(data['Stock_Excess_Vw_Ret']) + data['Unlevered_k']*data['Bond_inverse_sigma_hat']*(data['Bond_Excess_Vw_Ret'])
    levered_k = np.sqrt((np.std(data['Excess_Vw_Ret']))**2/(2*(1+data['Stock_Excess_Vw_Ret'].corr(data['Bond_Excess_Vw_Ret']))))
    data['Levered_k']= levered_k
    data['Excess_Levered_RP_Ret'] = (data['Levered_k']*data['Stock_inverse_sigma_hat']*(data['Stock_Excess_Vw_Ret'])) +  data['Bond_inverse_sigma_hat']*data['Levered_k']*(data['Bond_Excess_Vw_Ret'])

    Data = pd.DataFrame()
    Data = data[['year','month','Stock_Excess_Vw_Ret', 'Bond_Excess_Vw_Ret', 'Excess_Vw_Ret', 'Excess_60_40_Ret', 'Stock_inverse_sigma_hat',
             'Bond_inverse_sigma_hat', 'Unlevered_k', 'Excess_Unlevered_RP_Ret', 'Levered_k', 'Excess_Levered_RP_Ret']]
    return Data

In [12]:
Port_Rets = port_returns(Monthly_CRSP_Universe)
Port_Rets

,year,month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_inverse_sigma_hat,Bond_inverse_sigma_hat,Unlevered_k,Excess_Unlevered_RP_Ret,Levered_k,Excess_Levered_RP_Ret
0,1926,1,0.008054,-0.002951,0.008054,0.003652,NaN,NaN,NaN,NaN,0.039437,NaN
1,1926,2,0.073230,0.003386,0.073228,0.045293,NaN,NaN,NaN,NaN,0.039437,NaN
2,1926,3,0.051952,-0.002778,0.051950,0.030060,NaN,NaN,NaN,NaN,0.039437,NaN
3,1926,4,-0.010666,0.003082,-0.010665,-0.005167,NaN,NaN,NaN,NaN,0.039437,NaN
4,1926,5,0.049045,0.003539,0.049043,0.030843,NaN,NaN,NaN,NaN,0.039437,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1123,2019,8,0.006667,-0.010453,0.006658,-0.000181,18.232726,93.495024,0.008950,-0.007659,0.039437,-0.033746
1124,2019,9,0.055618,-0.018386,0.055580,0.026017,18.232906,91.586030,0.009106,-0.006099,0.039437,-0.026414
1125,2019,10,0.016498,0.013045,0.016496,0.015117,25.166734,88.414368,0.008804,0.013810,0.039437,0.061859
1126,2019,11,-0.048522,0.008376,-0.048493,-0.025763,25.876843,89.546917,0.008664,-0.004380,0.039437,-0.019936


In [13]:
def comparative_stats(data):
    data1 = Port_Rets.copy()
    data1 = data1[data1['year'] >=1930]
    data1 = data1[data1['year'] <= 2010]

    Annualized_Mean = [np.mean(data1['Stock_Excess_Vw_Ret'])*12, np.mean(data1['Bond_Excess_Vw_Ret'])*12, 
                       np.mean(data1['Excess_Vw_Ret'])*12, np.mean(data1['Excess_60_40_Ret'])*12, 
                       np.mean(data1['Excess_Unlevered_RP_Ret'])*12, np.mean(data1['Excess_Levered_RP_Ret'])*12]

    t_stat = [stats.ttest_1samp(data1['Stock_Excess_Vw_Ret'].fillna(0),0).statistic, 
              stats.ttest_1samp(data1['Bond_Excess_Vw_Ret'].fillna(0),0).statistic,
              stats.ttest_1samp(data1['Excess_Vw_Ret'].fillna(0),0).statistic,
              stats.ttest_1samp(data1['Excess_60_40_Ret'].fillna(0),0).statistic,
              stats.ttest_1samp(data1['Excess_Unlevered_RP_Ret'].fillna(0),0).statistic,
              stats.ttest_1samp(data1['Excess_Levered_RP_Ret'].fillna(0),0).statistic]

    Annualized_Standard_Deviation = [np.std(data1['Stock_Excess_Vw_Ret'])*np.sqrt(12), 
                                     np.std(data1['Bond_Excess_Vw_Ret'])*np.sqrt(12), 
                                     np.std(data1['Excess_Vw_Ret'])*np.sqrt(12),
                                     np.std(data1['Excess_60_40_Ret'])*np.sqrt(12), 
                                     np.std(data1['Excess_Unlevered_RP_Ret'])*np.sqrt(12), 
                                     np.std(data1['Excess_Levered_RP_Ret'])*np.sqrt(12)]
    
    skewness = [skew(data1['Stock_Excess_Vw_Ret'].fillna(0)), skew(data1['Bond_Excess_Vw_Ret'].fillna(0)), 
        skew(data1['Excess_Vw_Ret'].fillna(0)), skew(data1['Excess_60_40_Ret'].fillna(0)), 
        skew(data1['Excess_Unlevered_RP_Ret'].fillna(0)), skew(data1['Excess_Levered_RP_Ret'].fillna(0))] 
    
    Excess_Kurtosis = [kurtosis(data1['Stock_Excess_Vw_Ret'].fillna(0))-3, kurtosis(data1['Bond_Excess_Vw_Ret'].fillna(0))-3, 
                   kurtosis(data1['Excess_Vw_Ret'].fillna(0))-3, kurtosis(data1['Excess_60_40_Ret'].fillna(0))-3, 
                   kurtosis(data1['Excess_Unlevered_RP_Ret'].fillna(0))-3, kurtosis(data1['Excess_Levered_RP_Ret'].fillna(0))-3]
    
    Sharpe_Ratio = [x/y for x, y in zip(Annualized_Mean,Annualized_Standard_Deviation)]
    table1 = pd.DataFrame({
        'Annualized Mean': Annualized_Mean,
        't-stat of Annualized Mean': t_stat,
        'Annualized Standard Deviation': Annualized_Standard_Deviation,
        'Sharpe Ratio':Sharpe_Ratio,
        'Skewness':skewness,
        'Excess Kurtosis':Excess_Kurtosis
    }) 
    table1.index = ["CRSP stocks", "CRSP Bonds", "Value-weighted portfolio", "60/40 portfolio", "Unlevered RP", "Levered RP"]
    #table1['Sharpe Ratio'] = table1['Annualized Mean']/table1['Annualized Standard Deviation']
    table1['Annualized Mean'] <- table1['Annualized Mean']*100
    
    return(table1)

In [14]:
Final_Output = comparative_stats(Port_Rets)
Final_Output

,Annualized Mean,t-stat of Annualized Mean,Annualized Standard Deviation,Sharpe Ratio,Skewness,Excess Kurtosis
CRSP stocks,0.071691,3.440179,0.187165,0.383038,0.048857,5.240152
CRSP Bonds,0.006137,0.994343,0.055521,0.110539,-0.389704,-1.884430
Value-weighted portfolio,0.071659,3.440672,0.187055,0.383093,0.048767,5.243629
60/40 portfolio,0.045417,3.574057,0.114128,0.397945,0.049856,4.936135
Unlevered RP,0.028566,4.437215,0.054206,0.526993,-0.133891,-1.518975
Levered RP,0.130231,5.066118,0.216351,0.601943,-0.204894,-2.025866
